## 16.1
### 問
BinAdの命題"1+110+1=1000"に適用できるBinAdの推論規則を答えよ
### 答.
規則1 (A=1+110+1, B=1000), 規則3 (N=11)

In [5]:
import binAd
print(binAd.applyRuleOne("1+110+1=1000"))
print(binAd.applyRuleTwo("1+110+1=1000"))
print(binAd.applyRuleThree("1+110+1=1000"))
###ちょっと脱線###

print(binAd.isProvable("1+110+1=1000"))

{'1+1+110+1=1+1000'}
set()
{'111+1=1000'}
('not proved in first 109322 formulas generated', None)


## 16.6
### 問.
自分自身の論理系を作り, (a)正しいが証明不能な命題と(b)証明可能だが正しくない命題の例を挙げよ．
### 答.
- 証明系
    - アルファベット: 0~9までの数字と"/"と"->"  
    - 公理: "2020/01/01->2020/01/02" 
    - 推論規則: 
        1. 10進数に基づいて"次の数字"を定義する.
        2. "N->M"であればN, Mのそれぞれ最後の2桁を次の数字にしたものに変換可能  
            2-a. ただし, Mの真ん中の2桁が`(01|03|05|07|08|10)`で最後の2桁が`31`のとき, Mについては真ん中の2桁を次の数字に, 最後の2桁を`01`にしたものとする  
            2-b. ただし, Mの真ん中の2桁が`02`で最後の2桁が`28`のとき, Mについては真ん中の2桁を次の数字に, 最後の2桁を`01`にしたものとする  
            2-c. ただし, Mの真ん中の2桁が`(04|06|09|11)`で最後の2桁が`30`のとき, Mについては真ん中の2桁を次の数字に, 最後の2桁を`01`にしたものとする  
            2-d. ただし, Mの真ん中の2桁が`12`で最後の2桁が`31`のとき, Mについては最初の4桁を次の数字に, 真ん中の2桁を`01`に, 最後の2桁を`01`にしたものとする  
    - 適切な形式の命題を定義する規則: "N->N" ただし, Nは正規表現で`[0-9]{4}/[0-9]{2}/[0-9]{2}`を表す. 例"2000/01/01->2000/03/01"    
- 真理値割り当て: "N->M"の形式でN, Mが西暦の日付を表し,かつNの翌日がMであれば真

(a)正しいが証明不能な命題
"2020/02/28->2020/02/29"

(b)証明可能だが正しくない命題
"2020/02/28->2020/03/01"



## 16.11
### 問.
binAd.pyを出発点として論理系BrokenBinAdLogic, RestrictedBinAdLogic, FixedBinAdLogicを1つ以上実装せよ.不完全または不整合な論理系については, その不完全性, 不整合性を示す具体的なPythonコマンドを示せ．

In [11]:
from binAd import *
# 規則IBの実装
def applyRuleOneB(f):
    result = set()
    if not isWellFormed(f):
        return result
    (lhs,rhs) = f.split('=')
    # 右辺の'+1'を削除
    newF = '1+' + lhs + '=' + rhs
    result.add(newF)
    return result

# FixedBinAdLogicの真理値割り当ての実装
def isTrueFixed(f):
    if not isWellFormed(f):
        return False
    (lhs,rhs) = f.split('=')
    leftSum = computeBinarySum(lhs)
    rightSum = computeBinarySum(rhs)
    # 不等号に変更
    return (leftSum >= rightSum)

# RestrictedBinAdLogicのisProvableの実装
def applyRestrictedRules(f):
    return applyRuleOne(f) | applyRuleTwo(f)

def applyRestrictedRulesToFrontier(provedFormulas, frontierFormulas, predecessors = None):
    assert frontierFormulas <= provedFormulas
    newFrontier = set()
    # the ``sorted'' below is not needed for correctness, but is
    # needed to obtain same predecessors every time (there is
    # otherwise ambiguity in the definition)
    for f in sorted(frontierFormulas):
        newFs = applyRestrictedRules(f)
        newFrontier |= newFs
        if predecessors:
            # the ``sorted'' below is not needed for correctness, but is
            # needed to obtain same predecessors every time (there is
            # otherwise ambiguity in the definition)
            for newF in sorted(newFs): 
                if newF not in predecessors:
                    predecessors[newF] = f
    alreadyProved = newFrontier & provedFormulas
    newFrontier -= alreadyProved
    provedFormulas |= newFrontier
    return newFrontier

def isRestrictedProvable(f, maxFormulas = 100000):
    proof = None
    if not isWellFormed(f):
        result = 'not well-formed, therefore not provable'
    else:
        proved = set([theAxiom])
        frontier = proved
        predecessors = {theAxiom: ''}
        while len(proved)<maxFormulas:
            frontier = applyRestrictedRulesToFrontier(proved, frontier, predecessors)
            if f in proved:
                break
        if f in proved:
            result = 'provable'
            proof = getProofFromPredecessors(f, predecessors)
        else:
            result = 'not proved in first ' + str(len(proved)) + ' formulas generated'
    return (result, proof)

In [9]:
# BrokenBinAdLogicは公理と規則1, 1B, 2, 3とisTrueの真理値割り当てからなり, 不整合である
# 証明可能で偽である命題を示す.
print(applyRuleOneB('1=1'))
for i in applyRuleOneB('1=1'):
    print(isTrue(i))



{'1+1=1'}
False


In [12]:
# RestrictedBinAdLogicは公理と規則1, 2とisTrueの真理値割り当てからなり, 不完全である
# 証明不能で真である命題を示す.
print(isTrue('1+10=11'))
print(isRestrictedProvable('1+10=11'))

True
('not proved in first 154639 formulas generated', None)


## 16.16
### 問.
ゲーデルの不完全性定理に対する本書の証明は, 実際のPythonプログラムに大きく依存していた．本問と次問では, 自然言語の散文によるチューリングマシンの説明を使って証明を構成し直すための道筋を示す. まず, プログラム16.4のgodel.py のようなチューリングマシンMを定義する. マシンMは, 「Mが停止しないことを証明できるなら停止し, そうでなければループする」というアルゴリズムを実行する. 次に, godel.pyのnotHaltInPeanoのようなペアノ算術の文 S を定義する．S =「M は停止しない」とする. S は正しいが証明不能な命題であることを示せ．

### 答.
命題Sが証明可能であると仮定する. このとき, Mは停止するため, 命題Sと矛盾する. よってSは証明不能である.